### Referências:

https://medium.com/google-cloud/revolutionizing-code-reviews-using-generative-ai-a-practical-approach-using-google-cloud-vertex-ai-479e820c8e21

https://medium.com/@baristaGeek/how-to-properly-use-github-for-code-review-10d8caa38a39

https://mfadhel.com/github-code-reviews/

In [1]:
#pip install langchain openai gitpython faiss-cpu tqdm langchain-community transformers sentence-transformers huggingface_hub

In [2]:
#### Importando as libs:

import os
import shutil
import stat
from git import Repo
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.text_splitter import CharacterTextSplitter
from langchain.llms import HuggingFacePipeline
from langchain.chains import LLMChain
from transformers import pipeline

In [ ]:
# 1) Clonando o repo:

def clonar_repositorio(url_repositorio, caminho_local):
    if os.path.exists(caminho_local):
        def ao_remover_erro(func, caminho, exc_info):
            os.chmod(caminho, stat.S_IWRITE)
            func(caminho)
            shutil.rmtree(caminho_local, onerror=ao_remover_erro)
    Repo.clone_from(url_repositorio, caminho_local)
    print(f"Repositório clonado para {caminho_local}")

In [ ]:
# 2) Ler e processar os arquivos do repositório:
def preprocessar_repositorio(caminho_repositorio):
    conteudos_arquivos = []
    for raiz, _, arquivos in os.walk(caminho_repositorio):
        for arquivo in arquivos:
            if arquivo.endswith((".py", ".js", ".ts", ".java", ".c", ".cpp")):
                with open(os.path.join(raiz, arquivo), "r", encoding="utf-8", errors="ignore") as f:
                    conteudos_arquivos.append((arquivo, f.read()))
    return conteudos_arquivos

In [ ]:
# 3) Vetorizando o código
def vetorizar_codigo(conteudos_arquivos):
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    divisor_texto = CharacterTextSplitter(chunk_size=1000, chunk_overlap=100)

    textos = []
    metadados = []

    for nome_arquivo, conteudo in conteudos_arquivos:
        fragmentos = divisor_texto.split_text(conteudo)
        textos.extend(fragmentos)
        metadados.extend([{"source": nome_arquivo}] * len(fragmentos))

    loja_vetores = FAISS.from_texts(textos, embeddings, metadatas=metadados)
    return loja_vetores

In [ ]:
# 4) Verificando o código

def revisar_codigo(loja_vetores, pergunta):
        
    pipeline_revisao = pipeline("text-generation", model="Qwen/Qwen2.5-Coder-7B-Instruct")
    llm = HuggingFacePipeline(pipeline=pipeline_revisao)
    cadeia = LLMChain(llm=llm)

    resultados = loja_vetores.similarity_search(pergunta, k=5)

    contexto_revisao = "".join([f"Arquivo: {res.metadata['source']}\n{res.page_content}\n\n" for res in resultados])

    prompt = (
        f"You are a code review assistant. Analyze the following code snippets and provide feedback:\n\n"
        f"{contexto_revisao}\n\n"
        f"Feedback:"
    )

    resposta = cadeia.run(prompt)
    return resposta

: 

In [ ]:
def main():
    url_repositorio = "https://github.com/mopdc/gabpessoa.git"
    caminho_local = "./repositorio_clonado"

    print("Clonando repositório...")
    clonar_repositorio(url_repositorio, caminho_local)

    print("Processando repositório...")
    conteudos_arquivos = preprocessar_repositorio(caminho_local)

    print("Vetorizando código...")
    loja_vetores = vetorizar_codigo(conteudos_arquivos)

    print("Revisando código...")
    feedback = revisar_codigo(loja_vetores, "Identifique possíveis melhorias e problemas neste código.")

    print("\nFeedback da Revisão de Código:\n")
    print(feedback)

if __name__ == "__main__":
    main()

Cloning repository...
Repository cloned to ./cloned_repo
Processing repository...
Vectorizing code...


C:\Users\TRAVELER\AppData\Local\Temp\ipykernel_11448\2737378166.py:25: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
Created a chunk of size 1181, which is longer than the specified 1000
Created a chunk of size 1104, which is longer than the specified 1000
Created a chunk of size 3324, which is longer than the specified 1000


Reviewing code...
